In [1]:
import matplotlib as plt
print('matplotlib: {}'.format(plt.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn as sk
print('sklearn: {}'.format(sk.__version__))
from pandas_profiling import ProfileReport
import seaborn as sns
print('seaborn: {}'.format(sns.__version__))
import numpy as np
print('numpy: {}'.format(np.__version__))
from sklearn import linear_model
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
import xgboost as XGB
import lightgbm as lgb
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostRegressor


matplotlib: 3.3.2
pandas: 1.1.3
sklearn: 0.23.2
seaborn: 0.11.0
numpy: 1.19.2


In [2]:
X_train = pd.read_csv("X_train.csv")

In [3]:
X_val=pd.read_csv("X_val.csv")
X_test = pd.read_csv("X_test.csv")

In [4]:
y_train = pd.read_csv("y_train.csv")
y_val = pd.read_csv("y_val.csv")

In [2]:
sample_submission = pd.read_csv("sample_submission.csv")

In [3]:
id_test2 = sample_submission.Id

In [7]:
X_train = X_train.drop(columns=["Unnamed: 0"])

In [8]:
X_val = X_val.drop(columns=["Unnamed: 0"])

In [9]:
X_test = X_test.drop(columns=["Unnamed: 0"])

In [10]:
y_train =y_train.drop(columns=["Unnamed: 0"])

In [11]:
y_val = y_val.drop(columns=["Unnamed: 0"])

In [15]:
models = []
models.append(('LR', LinearRegression()))
models.append(('Ridge', linear_model.Ridge()))
models.append(('SVR', svm.SVR()))
models.append(('Lasso',linear_model.Lasso()))
models.append(('ElasticNet',ElasticNet()))
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold,scoring='neg_root_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: -0.135828 (0.008469)
Ridge: -0.134779 (0.007722)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

SVR: -0.336435 (0.017211)
Lasso: -0.302402 (0.020140)
ElasticNet: -0.299317 (0.020669)


In [19]:
# The best model is the ridge model

In [20]:
# Let's make a baseline

In [12]:
model = linear_model.Ridge()
model.fit(X_train, y_train)
predictions = model.predict(X_val)

In [13]:
mean_squared_error(y_val, predictions,squared=False)

0.14740600048182884

In [38]:
# Let's tune the hyperparameters

In [14]:
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
model = linear_model.Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas), scoring='neg_root_mean_squared_error')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.best_estimator_.alpha)


-0.13415896351556014
1.0


In [ ]:
# During EDA i took the log of the sale price to have a normally distributed feature
# Now I have to undo it to submit my results.

In [12]:
X_train2 = X_train.drop(columns="Condition2_RRAe")

In [16]:
model2 = linear_model.Ridge()
model2.fit(X_train2, y_train)

Ridge()

In [17]:
#model.fit(X_train, y_train)
predictions2 = model2.predict(X_test)


In [25]:
predictions2.shape

(1459, 1)

In [18]:
SalePrice =  np.exp(predictions2)

In [19]:
SalePrice = pd.DataFrame(SalePrice.flatten())

In [20]:
submission = pd.concat([id_test2,SalePrice],axis=1)

In [21]:
submission = submission.rename(columns={0:"SalePrice"})

In [110]:
submission.to_csv("submission1.csv")

In [111]:
# Let's export our first model

In [112]:
import joblib
joblib.dump(model2, 'ridge_model.pkl', compress=9)

['ridge_model.pkl']

In [121]:
# Let's try a more complex model on less features

In [4]:
X_train2 = pd.read_csv("X_train2.csv")
X_val2 = pd.read_csv("X_val2.csv")
X_test2 = pd.read_csv("X_test2.csv")
y_train2 = pd.read_csv("y_train2.csv")
y_val2 = pd.read_csv("y_val2.csv")

In [5]:
X_train2 = X_train2.drop(columns=["Unnamed: 0"])
X_val2 = X_val2.drop(columns=["Unnamed: 0"])
X_test2 = X_test2.drop(columns=["Unnamed: 0"])
y_train2 = y_train2.drop(columns=["Unnamed: 0"])
y_val2 = y_val2.drop(columns=["Unnamed: 0"])

In [136]:
models = []
models.append(('LR', LinearRegression()))
models.append(('Ridge', linear_model.Ridge()))
models.append(('SVR', svm.SVR()))
models.append(('Lasso',linear_model.Lasso()))
models.append(('ElasticNet',ElasticNet()))
models.append(('SGDRegressor',SGDRegressor()))
models.append(('DecisionTreeRegressor',DecisionTreeRegressor()))
models.append(('RandomForestRegressor',RandomForestRegressor()))
models.append(('KNNRegressor',KNeighborsRegressor()))
models.append(('GaussianProcessRegressor',GaussianProcessRegressor()))
models.append(('XGB',XGB.XGBRegressor()))
models.append(('LGBM',lgb.LGBMRegressor(objective='regression')))
models.append(('Adaboost',AdaBoostRegressor()))
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train2, y_train2, cv=kfold,scoring='neg_root_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: -0.146406 (0.009015)
Ridge: -0.146396 (0.008850)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

SVR: -0.332376 (0.017324)
Lasso: -0.302402 (0.020140)
ElasticNet: -0.299317 (0.020669)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

SGDRegressor: -2857874146114284.000000 (2272205583122694.500000)
DecisionTreeRegressor: -0.202453 (0.015683)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y wa

RandomForestRegressor: -0.153566 (0.008527)
KNNRegressor: -0.230470 (0.014101)
GaussianProcessRegressor: -6.723774 (1.274336)
XGB: -0.166595 (0.008012)
LGBM: -0.157355 (0.008907)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

Adaboost: -0.179476 (0.009124)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [135]:
# Let's try the Random Forest Regressor 

In [24]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [140]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train2, y_train2)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.3min finished
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [141]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 50,
 'bootstrap': True}

In [142]:
# Let's evaluate the best model from random search

In [145]:
model3 = RandomForestRegressor(n_estimators = 200,min_samples_split=2,min_samples_leaf=1,max_features = 'sqrt',
                              max_depth = 50,bootstrap=True)
model3.fit(X_train2, y_train2)
predictions3 = model3.predict(X_val2)

<ipython-input-145-b5add13e4154>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model3.fit(X_train2, y_train2)


In [146]:
mean_squared_error(y_val2, predictions3,squared=False)

0.16117424137580716

In [26]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [60, 70, 80, 90],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 4, 6],
    'n_estimators': [50, 100, 200, 500]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [152]:
grid_search.fit(X_train2, y_train2)
grid_search.best_params_

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   36.7s
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:   45.1s finished
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 90,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [153]:
model4 = RandomForestRegressor(n_estimators = 200,min_samples_split=2,min_samples_leaf=1,max_features = 'sqrt',
                              max_depth = 90,bootstrap=True)
model4.fit(X_train2, y_train2)
predictions4 = model4.predict(X_val2)

<ipython-input-153-861082e9e810>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model4.fit(X_train2, y_train2)


In [6]:
X_test2 = X_test2.drop(columns=["MSZoning_None"])

In [215]:
prediction5 = model4.predict(X_test2)

In [216]:
SalePrice3 =  np.exp(prediction5)

In [217]:
SalePrice3= pd.DataFrame(SalePrice3.flatten())
submission2 = pd.concat([id_test2,SalePrice3],axis=1)
#submission2 = submission.rename(columns={0:"SalePrice3"})

In [221]:
submission2.to_csv("submission2.csv")

In [7]:
X_train3 = X_train2.copy()
X_val3 = X_val2.copy()
X_test3 = X_test2.copy()

In [8]:
year_built_bins = np.linspace(1871, 2010, 10) # 10 bins
X_train3['YearBuilt'] = pd.cut(X_train3['YearBuilt'], bins=year_built_bins, labels=range(1, 10))
X_val3['YearBuilt'] = pd.cut(X_val3['YearBuilt'], bins=year_built_bins, labels=range(1, 10))
X_test3['YearBuilt']= pd.cut(X_test3['YearBuilt'], bins=year_built_bins, labels=range(1, 10))

In [9]:
year_built_bins2 = np.linspace(1949, 2010, 10) 
X_train3['YearRemodAdd'] = pd.cut(X_train3['YearRemodAdd'], bins=year_built_bins2, labels=range(1, 10))
X_val3['YearRemodAdd'] = pd.cut(X_val3['YearRemodAdd'], bins=year_built_bins2, labels=range(1, 10))
X_test3['YearRemodAdd']= pd.cut(X_test3['YearRemodAdd'], bins=year_built_bins2, labels=range(1, 10))


In [281]:
model5 = RandomForestRegressor()
model5.fit(X_train3, y_train2)
predictions6 = model5.predict(X_val3)

<ipython-input-281-a9175006ea81>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model5.fit(X_train3, y_train2)


In [283]:
mean_squared_error(y_val2, predictions6,squared=False)

0.1628527610530016

In [284]:
# Let's random search our new dataset

In [285]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [286]:
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train3, y_train2)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.1min finished
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'n_estimators': 2000,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [287]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [5, 10, 15, 20],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [1000, 2000, 5000, 10000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train3, y_train2)
grid_search.best_params_

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   42.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 12.5min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed: 15.8min finished
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 5000}

In [288]:
model6 = RandomForestRegressor(n_estimators = 5000,min_samples_split=2,min_samples_leaf=1,max_features = 'sqrt',
                              max_depth = 10,bootstrap=True)
model6.fit(X_train3, y_train2)
predictions7 = model6.predict(X_val3)

<ipython-input-288-bae3c299414a>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model6.fit(X_train3, y_train2)


In [289]:
mean_squared_error(y_val2, predictions7,squared=False)

0.15828270806441558

In [290]:
prediction8 = model6.predict(X_test3)

In [291]:
prediction8

array([11.65466968, 11.88352853, 12.08314157, ..., 11.91027182,
       11.57753859, 12.35810836])

In [292]:
SalePrice4 =  np.exp(prediction8)

In [302]:
submission3 =pd.concat([id_test2,pd.DataFrame(data=SalePrice4)],axis=1)

In [305]:
submission3 = submission3.rename(columns={0:"SalePrice"})

In [307]:
submission3.to_csv("submission3.csv")

In [308]:
# Let's try to stack the best performing models 

In [315]:
test =  RandomForestRegressor

In [11]:
estimators = [('LR',LinearRegression()),
             ('Ridge',linear_model.Ridge()),
             ('RandomForestRegressor',RandomForestRegressor),
             #('XGB',XGB.XGBRegressor()),
             ('LGBM',lgb.LGBMRegressor(objective='regression')),
             ('Adaboost',AdaBoostRegressor())
             ]


stack_model = StackingRegressor(estimators=estimators)


In [19]:
stack_model.fit(X_train3,y_train2)
prediction_stack = stack_model.predict(X_val3)
mean_squared_error(y_val2, prediction_stack,squared=False)

C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.

In [327]:
prediction_stack2 = stack_model.predict(X_test3)

In [328]:
prediction_stack2

array([11.55756434, 11.91274856, 12.02395643, ..., 11.93160851,
       11.6240442 , 12.37361015])

In [329]:
SalePrice5 =  np.exp(prediction_stack2)

In [330]:
submission4 =pd.concat([id_test2,pd.DataFrame(data=SalePrice5)],axis=1)

In [332]:
submission4 = submission4.rename(columns={0:"SalePrice"})

In [334]:
submission4.to_csv("submission4.csv")

In [336]:
stack_model.get_params()

{'cv': None,
 'estimators': [('LR', LinearRegression()),
  ('Ridge', Ridge()),
  ('RandomForestRegressor', RandomForestRegressor()),
  ('LGBM', LGBMRegressor(objective='regression')),
  ('Adaboost', AdaBoostRegressor())],
 'final_estimator': None,
 'n_jobs': None,
 'passthrough': False,
 'verbose': 0,
 'LR': LinearRegression(),
 'Ridge': Ridge(),
 'RandomForestRegressor': RandomForestRegressor(),
 'LGBM': LGBMRegressor(objective='regression'),
 'Adaboost': AdaBoostRegressor(),
 'LR__copy_X': True,
 'LR__fit_intercept': True,
 'LR__n_jobs': None,
 'LR__normalize': False,
 'Ridge__alpha': 1.0,
 'Ridge__copy_X': True,
 'Ridge__fit_intercept': True,
 'Ridge__max_iter': None,
 'Ridge__normalize': False,
 'Ridge__random_state': None,
 'Ridge__solver': 'auto',
 'Ridge__tol': 0.001,
 'RandomForestRegressor__bootstrap': True,
 'RandomForestRegressor__ccp_alpha': 0.0,
 'RandomForestRegressor__criterion': 'mse',
 'RandomForestRegressor__max_depth': None,
 'RandomForestRegressor__max_features': 'a

In [32]:
estimators = [('LR',LinearRegression()),
             ('Ridge',linear_model.Ridge()),
             ('RandomForestRegressor',RandomForestRegressor()),
             ('LGBM',lgb.LGBMRegressor(objective='regression')),
             ('Adaboost',AdaBoostRegressor())
             ]

stack_model2 = StackingRegressor(estimators=estimators)
params = {'Ridge__alpha': [1,0.1,0.01,0.001,0.0001],
         'RandomForestRegressor__max_depth':[10,25,50,100],
         'RandomForestRegressor__max_features': ['auto', 'sqrt', 'log2'],
         'RandomForestRegressor__n_estimators':[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
         'RandomForestRegressor__min_samples_leaf': [1, 2, 3],
         'RandomForestRegressor__min_samples_split': [2, 5, 10],
         'LGBM__boosting_type': ['gbdt','dart','rf','goss'],
         'LGBM__learning_rate': [0.001,0.005, 0.01,0.1,1],
         'LGBM__num_leaves': [31, 127],
         'LGBM__reg_alpha': [0.1, 0.3,0.5],
         'LGBM__reg_lambda': [0.1, 0.3,0.5],
         'LGBM__n_estimators': [50,100,200],
         'Adaboost__learning_rate': [0.01,0.05,0.1,0.3,1],
         'Adaboost__loss': ['linear', 'square', 'exponential'],
         'Adaboost__n_estimators':[50, 100]
         }
grid2 = RandomizedSearchCV(estimator=stack_model2, param_distributions=params, cv=5,scoring='neg_root_mean_squared_error')
grid2.fit(X_train3, y_train2)

C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

RandomizedSearchCV(cv=5,
                   estimator=StackingRegressor(estimators=[('LR',
                                                            LinearRegression()),
                                                           ('Ridge', Ridge()),
                                                           ('RandomForestRegressor',
                                                            RandomForestRegressor()),
                                                           ('LGBM',
                                                            LGBMRegressor(objective='regression')),
                                                           ('Adaboost',
                                                            AdaBoostRegressor())]),
                   param_distributions={'Adaboost__learning_rate': [0.01, 0.05,
                                                                    0.1, 0.3,
                                                                    1],
                        

In [33]:
grid2.best_params_

{'Ridge__alpha': 1,
 'RandomForestRegressor__n_estimators': 600,
 'RandomForestRegressor__min_samples_split': 2,
 'RandomForestRegressor__min_samples_leaf': 1,
 'RandomForestRegressor__max_features': 'auto',
 'RandomForestRegressor__max_depth': 10,
 'LGBM__reg_lambda': 0.5,
 'LGBM__reg_alpha': 0.1,
 'LGBM__num_leaves': 127,
 'LGBM__n_estimators': 100,
 'LGBM__learning_rate': 0.001,
 'LGBM__boosting_type': 'dart',
 'Adaboost__n_estimators': 50,
 'Adaboost__loss': 'exponential',
 'Adaboost__learning_rate': 0.3}

In [34]:
estimators2 = [('LR',LinearRegression()),
             ('Ridge',linear_model.Ridge(alpha=1)),
             ('RandomForestRegressor',RandomForestRegressor(n_estimators=600,min_samples_split=2,min_samples_leaf =1,
                                     max_features='auto',max_depth=10 )),
             ('LGBM',lgb.LGBMRegressor(objective='regression',reg_lambda=0.5,reg_alpha=0.1,num_leaves=127,
                                       n_estimators=100,learning_rate=0.001,boosting_type='dart')),
             ('Adaboost',AdaBoostRegressor(n_estimators=50,loss='exponential',learning_rate=0.3))
             ]

stack_model2 = StackingRegressor(estimators=estimators)

In [35]:
stack_model2.fit(X_train3,y_train2)
prediction_stack3 = stack_model2.predict(X_val3)
mean_squared_error(y_val2, prediction_stack3,squared=False)

C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.1537386193800115

In [36]:
prediction_stack4 = stack_model2.predict(X_test3)

In [38]:
SalePrice6 =  np.exp(prediction_stack4)
submission5 = pd.concat([id_test2,pd.DataFrame(data=SalePrice6)],axis=1)
submission5 = submission5.rename(columns={0:"SalePrice"})

In [40]:
submission5.to_csv("submission5.csv")